# Scraper script OpenMinVWS dossiers
Gebruikt Selenium om de dossier batch id te vinden en leidt naar de download pagina. BeutifulSoup download vervolgens het dossier.

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

# 🔹 **Selenium functie om batch-ID op te halen**
def find_batch_id(dossier_url):
    """
    Opent een dossierpagina, klikt op de downloadknop en retourneert de batch-URL.
    """
    chrome_driver_path = "/usr/local/bin/chromedriver"
    service = Service(chrome_driver_path)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Zet op False als je de browser wilt zien
    driver = webdriver.Chrome(service=service, options=options)

    try:
        driver.get(dossier_url)
        time.sleep(3)  # Wacht zodat JavaScript kan laden

        download_button = driver.find_element(By.XPATH, "//button[@data-e2e-name='download-documents-button']")
        download_button.click()
        time.sleep(3)  # Wacht tot de nieuwe pagina met batch-ID laadt

        batch_url = driver.current_url
        return batch_url + "/download"

    except Exception as e:
        print(f"❌ Fout bij vinden van batch-ID: {e}")
        return None
    finally:
        driver.quit()

# Voorbeeldgebruik:
# dossier_url = "https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz"
# batch_url = find_batch_id(dossier_url)
# print(f"✅ Batch-URL: {batch_url}")


In [ ]:
# 🔹 **Download link voor besluitbrief**
def get_document_download_link(document_page_url):
    """
    Haalt de directe downloadlink van de besluitbrief op.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(document_page_url, headers=headers)
    if response.status_code != 200:
        print(f"❌ Kan de documentpagina niet bereiken: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    download_link_element = soup.find("a", {"data-e2e-name": "download-file-link"})
    return "https://open.minvws.nl" + download_link_element["href"] if download_link_element else None

In [ ]:
import requests
from bs4 import BeautifulSoup

# 🔹 **Metadata en besluitbrief scraper**
def get_dossier_metadata_and_document(dossier_url):
    """
    Scraped metadata en de link naar de besluitbrief.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(dossier_url, headers=headers)
    if response.status_code != 200:
        print(f"❌ Kan de pagina niet bereiken: {response.status_code}")
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")

    title_element = soup.find("h1", {"data-e2e-name": "dossier-metadata-title"})
    dossier_title = title_element.get_text(strip=True) if title_element else "Niet gevonden"
    if ":" in dossier_title:
        dossier_title = dossier_title.split(":", 1)[1].strip()

    published_date_element = soup.find("dd", {"data-e2e-name": "dossier-metadata-published-date"})
    published_date = published_date_element.find("time")["datetime"] if published_date_element else "Niet gevonden"

    decision_date_element = soup.find("td", {"data-e2e-name": "dossier-metadata-decision-date"})
    decision_date = decision_date_element.find("time")["datetime"] if decision_date_element else "Niet gevonden"

    document_link_element = soup.find("a", {"data-e2e-name": "main-document-detail-link"})
    document_page_url = "https://open.minvws.nl" + document_link_element["href"] if document_link_element else None

    return {"title": dossier_title, "published_date": published_date, "decision_date": decision_date}, document_page_url
    
# Voorbeeldgebruik
# dossier_url = "https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz"
# metadata = get_dossier_metadata(dossier_url)

# print("📂 Dossier Metadata:")
# print(f"🔹 Titel: {metadata['title']}")
# print(f"📅 Publicatiedatum: {metadata['published_date']}")
# print(f"📜 Beslissingsdatum: {metadata['decision_date']}")


In [ ]:
import requests
import json
import zipfile
import os

# 🔹 **ZIP-bestand + besluitbrief downloaden**
def download_zip_and_document(dossier_url, save_path):
    """
    Zoek de ZIP-downloadlink met Selenium, download ZIP en besluitbrief en voeg metadata toe.
    """
    zip_url = find_batch_id(dossier_url)
    if not zip_url:
        print("❌ Geen batch-ID gevonden, ZIP wordt niet gedownload.")
        return

    metadata, document_page_url = get_dossier_metadata_and_document(dossier_url)
    if not metadata:
        print("⚠️ Geen metadata gevonden, ZIP wordt zonder metadata opgeslagen.")
        return

    zip_response = requests.get(zip_url, stream=True)
    if zip_response.status_code != 200:
        print(f"❌ Fout bij downloaden van ZIP: {zip_response.status_code}")
        return

    temp_zip_path = save_path + ".temp.zip"
    with open(temp_zip_path, "wb") as f:
        for chunk in zip_response.iter_content(1024):
            f.write(chunk)

    print(f"✅ ZIP gedownload: {temp_zip_path}")

    metadata_json = json.dumps(metadata, indent=4)
    safe_title = sanitize_filename(metadata['title'])
    final_zip_path = os.path.join(save_path, safe_title + ".zip")
    
    with zipfile.ZipFile(temp_zip_path, 'a') as zipf:
        zipf.writestr("metadata.json", metadata_json)

    if document_page_url:
        print(f"📄 Bezoek besluitbrief-pagina: {document_page_url}")
        document_download_url = get_document_download_link(document_page_url)

        if document_download_url:
            doc_response = requests.get(document_download_url, stream=True)
            if doc_response.status_code == 200:
                temp_doc_path = save_path + "_besluitbrief.pdf"
                with open(temp_doc_path, "wb") as f:
                    for chunk in doc_response.iter_content(1024):
                        f.write(chunk)

                print(f"✅ Besluitbrief gedownload: {temp_doc_path}")

                with zipfile.ZipFile(temp_zip_path, 'a') as zipf:
                    zipf.write(temp_doc_path, arcname="besluitbrief.pdf")

                os.remove(temp_doc_path)
                print(f"📂 Besluitbrief toegevoegd aan ZIP: {final_zip_path}")

    os.rename(temp_zip_path, final_zip_path)
    print(f"✅ ZIP compleet: {final_zip_path}")

    time.sleep(5)
    
    
# Voorbeeldgebruik:
# dossier_url = "https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz"
# save_path = f"/home/nena-meijer/PyCharmMiscProject/dossiers_zip/{metadata['title']}"

# download_zip_and_document(dossier_url, save_path)

In [ ]:
def sanitize_filename(title, max_length=150):
    """
    Zorgt ervoor dat de bestandsnaam niet te lang wordt.
    """
    title = title.replace("/", "-")  # Vermijd ongeldige tekens
    return title[:max_length].strip()  # Knip af tot de max lengte

In [ ]:
# Pad naar het bestand met de dossier-URLs
dossier_urls_path = "/home/nena-meijer/Downloads/dossier_urls.txt"
save_path = "/home/nena-meijer/PyCharmMiscProject/dossiers_zip/"

# Lees alle dossiers uit het bestand
with open(dossier_urls_path, "r") as f:
    dossier_urls = [line.strip() for line in f.readlines()][246:]
    
start_dossier_nummer = 246

for dossier_url in dossier_urls:
    print(f"Dossiernummer: {start_dossier_nummer}")
    print(f"📂 Start met downloaden van dossier: {dossier_url}")
    download_zip_and_document(dossier_url, save_path)
    print("✅ Download compleet. Start volgende...")
    start_dossier_nummer += 1

print("🎉 Alle dossiers zijn gedownload!")


In [ ]:
# URL's matchen aan dossiers

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from IPython.display import display

# CSV met bestaande dossiers inlezen
df = pd.read_csv("old_csvs/df_dossiers.csv")

# URLs inlezen uit dossier_urls.txt
with open("dossier_urls.txt", "r") as f:
    dossier_urls = [line.strip() for line in f.readlines()]

# Functie om de metadata titel van een dossierpagina op te halen
def get_dossier_metadata_title(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"❌ Kan de pagina niet laden: {url} (Status: {response.status_code})")
        return None

    soup = BeautifulSoup(response.text, "html.parser")

    # Zoek de metadata titel van het dossier
    title_element = soup.find("h1", {"data-e2e-name": "dossier-metadata-title"})
    if title_element:
        title_text = title_element.get_text(strip=True).replace("Besluit over:", "").strip()
        return title_text
    return None

# Voeg een kolom voor de URL toe aan de DataFrame
df["dossier_sourceURL"] = None

# Loop door de URLs en zoek matches
for url in dossier_urls:
    dossier_title = get_dossier_metadata_title(url)

    if dossier_title:
        match_idx = df[df["dossier_name"] == dossier_title].index
        if not match_idx.empty:
            df.loc[match_idx, "dossier_sourceURL"] = url
            print(f"✅ Match gevonden: {dossier_title} → {url}")

    time.sleep(1)  # 🔹 Wacht 2 seconden om blokkering te voorkomen

# Opslaan als nieuwe CSV
df.to_csv("df_dossiers.csv", index=False)

# Weergeven in Jupyter Notebook
display(df)

print("✅ Dossier-URLs toegevoegd en opgeslagen als 'df_dossiers.csv'")


In [5]:
# summary en subject toevoegen aan dossiers
import requests
from bs4 import BeautifulSoup
import time

def scrape_dossier_metadata(dossier_url):
    """
    Haalt de samenvatting (dossier_summary) en het onderwerp (dossier_subject) op van een dossierpagina.

    Parameters:
    dossier_url (str): De URL van de dossierpagina.

    Returns:
    tuple: (dossier_summary, dossier_subject)
    """
    headers = {"User-Agent": "Mozilla/5.0"}

    try:
        # Haal de HTML-pagina op
        response = requests.get(dossier_url, headers=headers)
        if response.status_code != 200:
            print(f"❌ Kan {dossier_url} niet bereiken, status {response.status_code}")
            return None, None

        time.sleep(1)

        # Parse de HTML met BeautifulSoup
        soup = BeautifulSoup(response.text, "html.parser")

        # Scrape dossier_summary
        summary_element = soup.find("p", {"data-e2e-name": "dossier-summary"})
        dossier_summary = summary_element.get_text(strip=True) if summary_element else None

        # Scrape dossier_subject (eerste gevonden onderwerp)
        subject_element = soup.find("p").find("a", class_="woo-a") if soup.find("p") else None
        dossier_subject = subject_element.get_text(strip=True) if subject_element else None

        return dossier_summary, dossier_subject

    except Exception as e:
        print(f"⚠️ Fout bij verwerken van {dossier_url}: {e}")
        return None, None

In [6]:
import pandas as pd

# Laad df_dossiers.csv
df_dossiers = pd.read_csv("old_csvs/df_dossiers.csv")

# Voeg lege kolommen toe voor de nieuwe data (indien ze nog niet bestaan)
if "dossier_summary" not in df_dossiers.columns:
    df_dossiers["dossier_summary"] = None
if "dossier_subject" not in df_dossiers.columns:
    df_dossiers["dossier_subject"] = None

# Lees de URLs uit dossier_urls.txt
with open("dossier_urls.txt", "r") as file:
    dossier_urls = [line.strip() for line in file.readlines()]

# Loop door alle URLs en scrap de metadata
for index, row in df_dossiers.iterrows():
    dossier_url = row["dossier_sourceURL"]

    # Alleen scrapen als de URL in dossier_urls.txt voorkomt
    if dossier_url in dossier_urls:
        summary, subject = scrape_dossier_metadata(dossier_url)

        # Update DataFrame
        df_dossiers.at[index, "dossier_summary"] = summary
        df_dossiers.at[index, "dossier_subject"] = subject

# Opslaan als nieuw CSV-bestand
df_dossiers.to_csv("df_dossiers_updated.csv", index=False)

# Toon de geüpdatete DataFrame
from IPython.display import display
display(df_dossiers)

print("✅ Scraping voltooid en opgeslagen als 'df_dossiers_updated.csv'")


,dossier_id,dossier_name,dossier_published_date,dossier_decision_date,document_count,dossier_sourceURL,dossier_summary,dossier_subject
0,1,(4e deel) Besluit Woo-verzoek Covid-19 aanpak ...,2024-02-05,2023-07-29,229,https://open.minvws.nl/dossier/VWS-WC/098,Besluit\r\nIk heb besloten om een deel van de ...,None
1,2,Woo-besluit aangaande kaders werkproces Wob-ve...,2024-03-22,2023-03-20,83,https://open.minvws.nl/dossier/VWS-WOO/3518578...,"De minister van Volksgezondheid, Welzijn en Sp...",None
2,3,1e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-07,2022-08-18,208,https://open.minvws.nl/dossier/VWS-WC/090,Besluit\r\nIk heb besloten om een deel van de ...,None
3,4,2e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-08,2023-02-17,239,https://open.minvws.nl/dossier/VWS-WC/095,Besluit\r\nIk heb besloten om een deel van de ...,None
4,5,3e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-07,2022-05-12,144,https://open.minvws.nl/dossier/VWS-WC/096,Besluit\r\nIk heb besloten om een deel van de ...,None
...,...,...,...,...,...,...,...,...
215,216,Woo-deelbesluit aangaande overeenkomsten PCR-l...,2024-06-06,2024-03-27,68,https://open.minvws.nl/dossier/VWS-WOO/3792667...,"De minister van Volksgezondheid, Welzijn en Sp...",None
216,217,Woo-deelbesluit aangaande vaccinatie aansprake...,2024-10-09,2024-10-02,116,https://open.minvws.nl/dossier/VWS-WOO/3939665...,"De minister van Volksgezondheid, Welzijn en Sp...",None
217,218,Woo-deelbesluit aangaande ‘Capaciteit van ziek...,2024-10-19,2024-09-12,199,https://open.minvws.nl/dossier/VWS-WOO/3959811...,"De minister van Volksgezondheid, Welzijn en Sp...",None
218,219,Woo-deelbesluit aangaande ‘de inkoop van perso...,2024-04-05,2024-02-14,2695,https://open.minvws.nl/dossier/VWS-WOO/3773352...,"De minister van Volksgezondheid, Welzijn en Sp...",None


✅ Scraping voltooid en opgeslagen als 'df_dossiers_updated.csv'


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from IPython.display import display

# CSV met bestaande dossiers inlezen
df = pd.read_csv("old_csvs/df_dossiers.csv")

# URLs inlezen uit dossier_urls.txt
with open("dossier_urls.txt", "r") as f:
    dossier_urls = [line.strip() for line in f.readlines()]

# Functie om de dossiernaam en verantwoordelijke instantie op te halen
def get_dossier_info(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"❌ Kan de pagina niet laden: {url} (Status: {response.status_code})")
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")

    # Dossiernaam ophalen en 'Besluit over:' verwijderen
    title_element = soup.find("h1", {"data-e2e-name": "dossier-metadata-title"})
    dossier_name = title_element.get_text(strip=True).replace("Besluit over:", "").strip() if title_element else None

    # Verantwoordelijke instantie ophalen
    responsible_element = soup.find("td", {"data-e2e-name": "dossier-metadata-responsible"})
    responsible = responsible_element.get_text(strip=True) if responsible_element else None

    return dossier_name, responsible

# Voeg een kolom voor de verantwoordelijke instantie toe aan de DataFrame (als die nog niet bestaat)
if "dossier_responsible" not in df.columns:
    df["dossier_responsible"] = None

# Loop door de URLs en zoek de dossiernaam + verantwoordelijke instantie
for url in dossier_urls:
    dossier_name, responsible = get_dossier_info(url)

    if dossier_name and responsible:
        match_idx = df[df["dossier_name"] == dossier_name].index
        if not match_idx.empty:
            df.loc[match_idx, "dossier_responsible"] = responsible
            print(f"✅ Match gevonden: {dossier_name} → {responsible}")

    time.sleep(1)  # 🔹 Wacht 1 seconde om blokkering te voorkomen

# Opslaan als nieuwe CSV
df.to_csv("df_dossiers_responsible.csv", index=False)

# Weergeven in Jupyter Notebook
display(df)

print("✅ Verantwoordelijke instanties toegevoegd en opgeslagen als 'df_dossiers_responsible.csv'")


✅ Match gevonden: Wob-deelbesluit aangaande Vaccinaties en medicatie over de periode februari 2020 → ministerie van Volksgezondheid, Welzijn en Sport
✅ Match gevonden: Wob-deelbesluit aangaande Digitale Middelen over de periode september 2020 → ministerie van Volksgezondheid, Welzijn en Sport
✅ Match gevonden: Woo-besluit aangaande kaders werkproces Wob-verzoeken → ministerie van Volksgezondheid, Welzijn en Sport
✅ Match gevonden: Woo-deelbesluit aangaande Overleg VWS over de periode juni, juli en augustus 2021 → ministerie van Volksgezondheid, Welzijn en Sport
✅ Match gevonden: Woo-deelbesluit aangaande Overleggen Overige over de periode september 2020 → ministerie van Volksgezondheid, Welzijn en Sport
✅ Match gevonden: Woo-besluit chatberichten aangaande communicatie tussen minister Hugo de Jonge en Jaap van Dissel, tussen minister Hugo de Jonge en minister-president Mark Rutte en tussen Jaap van Dissel en minister-president Mark Rutte die betrekking heeft op de bestrijding van het c

,dossier_id,dossier_name,dossier_published_date,dossier_decision_date,document_count,dossier_sourceURL,dossier_responsible
0,1,(4e deel) Besluit Woo-verzoek Covid-19 aanpak ...,2024-02-05,2023-07-29,229,https://open.minvws.nl/dossier/VWS-WC/098,ministerie van Justitie en Veiligheid
1,2,Woo-besluit aangaande kaders werkproces Wob-ve...,2024-03-22,2023-03-20,83,https://open.minvws.nl/dossier/VWS-WOO/3518578...,"ministerie van Volksgezondheid, Welzijn en Sport"
2,3,1e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-07,2022-08-18,208,https://open.minvws.nl/dossier/VWS-WC/090,ministerie van Justitie en Veiligheid
3,4,2e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-08,2023-02-17,239,https://open.minvws.nl/dossier/VWS-WC/095,ministerie van Justitie en Veiligheid
4,5,3e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-07,2022-05-12,144,https://open.minvws.nl/dossier/VWS-WC/096,ministerie van Justitie en Veiligheid
...,...,...,...,...,...,...,...
215,216,Woo-deelbesluit aangaande overeenkomsten PCR-l...,2024-06-06,2024-03-27,68,https://open.minvws.nl/dossier/VWS-WOO/3792667...,"ministerie van Volksgezondheid, Welzijn en Sport"
216,217,Woo-deelbesluit aangaande vaccinatie aansprake...,2024-10-09,2024-10-02,116,https://open.minvws.nl/dossier/VWS-WOO/3939665...,"ministerie van Volksgezondheid, Welzijn en Sport"
217,218,Woo-deelbesluit aangaande ‘Capaciteit van ziek...,2024-10-19,2024-09-12,199,https://open.minvws.nl/dossier/VWS-WOO/3959811...,"ministerie van Volksgezondheid, Welzijn en Sport"
218,219,Woo-deelbesluit aangaande ‘de inkoop van perso...,2024-04-05,2024-02-14,2695,https://open.minvws.nl/dossier/VWS-WOO/3773352...,"ministerie van Volksgezondheid, Welzijn en Sport"


✅ Verantwoordelijke instanties toegevoegd en opgeslagen als 'df_dossiers_responsible.csv'


In [2]:
# extract subject per dossier

import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from IPython.display import display

# CSV met bestaande dossiers inlezen
df = pd.read_csv("old_csvs/df_dossiers_responsible.csv")

# URLs inlezen uit dossier_urls.txt
with open("dossier_urls.txt", "r") as f:
    dossier_urls = [line.strip() for line in f.readlines()]

# Functie om de dossiernaam en onderwerp op te halen
def get_dossier_subject(url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"❌ Kan de pagina niet laden: {url} (Status: {response.status_code})")
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")

    # Dossiernaam ophalen en 'Besluit over:' verwijderen
    title_element = soup.find("h1", {"data-e2e-name": "dossier-metadata-title"})
    dossier_name = title_element.get_text(strip=True).replace("Besluit over:", "").strip() if title_element else None

    # Onderwerp ophalen uit de bijbehorende <a> tags onder <th> met "Onderwerp"
    subject_element = soup.find("th", string="Onderwerp")
    subject = None
    if subject_element:
        subject_td = subject_element.find_next("td")  # Zoek de volgende <td> eronder
        if subject_td:
            subject_links = subject_td.find_all("a", class_="woo-a")
            subject = ", ".join(link.get_text(strip=True) for link in subject_links) if subject_links else None

    return dossier_name, subject

# Voeg een kolom voor dossier_subject toe aan de DataFrame als die nog niet bestaat
if "dossier_subject" not in df.columns:
    df["dossier_subject"] = None

# Loop door de URLs en zoek de dossiernaam en onderwerp
for url in dossier_urls:
    dossier_name, subject = get_dossier_subject(url)

    if dossier_name and subject:
        match_idx = df[df["dossier_name"] == dossier_name].index
        if not match_idx.empty:
            df.loc[match_idx, "dossier_subject"] = subject
            print(f"✅ Match gevonden: {dossier_name} → Onderwerp: {subject}")

    time.sleep(1)  # 🔹 Wacht 1 seconde om blokkering te voorkomen

# Opslaan als nieuwe CSV
df.to_csv("df_dossiers_final.csv", index=False)

# Weergeven in Jupyter Notebook
display(df)

print("✅ Onderwerpen toegevoegd en opgeslagen als 'df_dossiers_final.csv'")


✅ Match gevonden: Wob-deelbesluit aangaande Vaccinaties en medicatie over de periode februari 2020 → Onderwerp: Vaccinaties en medicatie
✅ Match gevonden: Wob-deelbesluit aangaande Digitale Middelen over de periode september 2020 → Onderwerp: Digitale middelen
✅ Match gevonden: Woo-besluit aangaande kaders werkproces Wob-verzoeken → Onderwerp: Scenario’s en maatregelen
✅ Match gevonden: Woo-deelbesluit aangaande Overleg VWS over de periode juni, juli en augustus 2021 → Onderwerp: Overleg VWS
✅ Match gevonden: Woo-deelbesluit aangaande Overleggen Overige over de periode september 2020 → Onderwerp: Overleg overig
✅ Match gevonden: Woo-besluit chatberichten aangaande communicatie tussen minister Hugo de Jonge en Jaap van Dissel, tussen minister Hugo de Jonge en minister-president Mark Rutte en tussen Jaap van Dissel en minister-president Mark Rutte die betrekking heeft op de bestrijding van het coronavirus in de meest brede zin → Onderwerp: Chats
✅ Match gevonden: Woo-deelbesluit aangaand

,dossier_id,dossier_name,dossier_published_date,dossier_decision_date,document_count,dossier_sourceURL,dossier_responsible,dossier_subject
0,1,(4e deel) Besluit Woo-verzoek Covid-19 aanpak ...,2024-02-05,2023-07-29,229,https://open.minvws.nl/dossier/VWS-WC/098,ministerie van Justitie en Veiligheid,Overleg overig
1,2,Woo-besluit aangaande kaders werkproces Wob-ve...,2024-03-22,2023-03-20,83,https://open.minvws.nl/dossier/VWS-WOO/3518578...,"ministerie van Volksgezondheid, Welzijn en Sport",Scenario’s en maatregelen
2,3,1e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-07,2022-08-18,208,https://open.minvws.nl/dossier/VWS-WC/090,ministerie van Justitie en Veiligheid,Overleg overig
3,4,2e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-08,2023-02-17,239,https://open.minvws.nl/dossier/VWS-WC/095,ministerie van Justitie en Veiligheid,Overleg overig
4,5,3e deel Besluit Woo verzoek Covid-19 aanpak AC...,2024-02-07,2022-05-12,144,https://open.minvws.nl/dossier/VWS-WC/096,ministerie van Justitie en Veiligheid,Overleg overig
...,...,...,...,...,...,...,...,...
215,216,Woo-deelbesluit aangaande overeenkomsten PCR-l...,2024-06-06,2024-03-27,68,https://open.minvws.nl/dossier/VWS-WOO/3792667...,"ministerie van Volksgezondheid, Welzijn en Sport",Testen
216,217,Woo-deelbesluit aangaande vaccinatie aansprake...,2024-10-09,2024-10-02,116,https://open.minvws.nl/dossier/VWS-WOO/3939665...,"ministerie van Volksgezondheid, Welzijn en Sport",Vaccinaties en medicatie
217,218,Woo-deelbesluit aangaande ‘Capaciteit van ziek...,2024-10-19,2024-09-12,199,https://open.minvws.nl/dossier/VWS-WOO/3959811...,"ministerie van Volksgezondheid, Welzijn en Sport",Capaciteit ziekenhuizen
218,219,Woo-deelbesluit aangaande ‘de inkoop van perso...,2024-04-05,2024-02-14,2695,https://open.minvws.nl/dossier/VWS-WOO/3773352...,"ministerie van Volksgezondheid, Welzijn en Sport",Overleg VWS


✅ Onderwerpen toegevoegd en opgeslagen als 'df_dossiers_subject.csv'


In [14]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin
from IPython.display import display

# CSV-bestanden inlezen
df_dossiers = pd.read_csv("df_dossiers_final.csv")
df_documents = pd.read_csv("df_documents.csv")

# Base URL van de site
BASE_URL = "https://open.minvws.nl"

# Mapping van <use> symbolen naar documenttypen
DOCUMENT_TYPE_MAP = {
    "unknown": "Onbekend",
    "pdf": "PDF",
    "email": "E-mailbericht",
    "presentation": "Presentatie",
    "doc": "Word-document",
    "chat": "Chatbericht",
    "image": "Afbeelding",
    "spreadsheet": "Spreadsheet"
}

# Voeg kolommen toe als ze nog niet bestaan
if "document_sourcetype" not in df_documents.columns:
    df_documents["document_sourcetype"] = None
if "document_date" not in df_documents.columns:
    df_documents["document_date"] = None

# Functie om document metadata te scrapen uit een dossierpagina
def get_document_metadata(url, dossier_id):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"❌ Kan de pagina niet laden: {url} (Status: {response.status_code})")
        return []

    soup = BeautifulSoup(response.text, "html.parser")

    # Documenten verzamelen
    document_rows = soup.select("div[data-e2e-name='documents-section'] table tbody tr")

    metadata_list = []

    for row in document_rows:
        # Documentnaam ophalen en opschonen
        document_name_element = row.select_one("td div.woo-readable-width a.woo-a")
        raw_document_name = document_name_element.get_text(strip=True) if document_name_element else None
        document_name = raw_document_name.replace("_", " ") if raw_document_name else None  # Vervang underscores

        # **Besluitbrief overslaan bij scrapen**
        if document_name and "besluitbrief.pdf" in document_name.lower():
            continue  # Sla besluitbrief.pdf over

        # Documentdatum ophalen
        document_date_element = row.select_one("td time")
        document_date = document_date_element["datetime"] if document_date_element else None

        # Documenttype ophalen uit de <use> of <symbol>
        document_type_element = row.select_one("td svg use")
        document_type = None
        if document_type_element:
            type_symbol = document_type_element["xlink:href"].split("#")[-1]
            document_type = DOCUMENT_TYPE_MAP.get(type_symbol, "Onbekend")

        # Alleen opslaan als er een documentnaam is
        if document_name:
            metadata_list.append({
                "dossier_id": dossier_id,
                "document_title_clean": document_name,  # Schoon document_title (spaties i.p.v. underscores)
                "document_sourcetype": document_type,
                "document_date": document_date
            })

    return metadata_list

# Filter het dossier met dossier_id = 8
df_dossier_8 = df_dossiers[df_dossiers["dossier_id"] == 8]

# Lijst om document-metadata op te slaan
all_metadata = []

# Loop door het dossier (maar nu alleen voor dossier_id=8)
for _, row in df_dossier_8.iterrows():
    dossier_id = row["dossier_id"]
    dossier_url = row["dossier_sourceURL"]

    print(f"🔍 Scrapen van dossier: {dossier_id} → {dossier_url}")

    # Haal metadata van documenten op
    metadata = get_document_metadata(dossier_url, dossier_id)
    all_metadata.extend(metadata)

    time.sleep(1)  # Voorkom blokkering

# Zet de metadata in een DataFrame
df_scraped_metadata = pd.DataFrame(all_metadata)

# 🔹 **Substring-matching met `_` vervangen door spaties**
def match_document_title(scraped_title, df_docs):
    """Zoekt of `scraped_title` als substring voorkomt in `df_docs["document_title"]` (met underscores vervangen door spaties)."""
    scraped_title_clean = scraped_title.replace("_", " ")
    matches = df_docs[df_docs["document_title"].str.replace("_", " ").str.contains(scraped_title_clean, case=False, na=False)]
    if not matches.empty:
        return matches.iloc[0]["document_title"]  # Neem de eerste match
    return None  # Geen match gevonden

# **Filter df_documents zodat alleen documenten van dossier_id = 8 worden meegenomen en besluitbrief.pdf wordt overgeslagen**
df_documents_8 = df_documents[
    (df_documents["dossier_id"] == 8) &
    (~df_documents["document_title"].str.contains("besluitbrief.pdf", case=False, na=False))
].copy()

# Voor elk gescrapet document, zoeken naar een match in df_documents
matched_rows = []
for _, scraped_row in df_scraped_metadata.iterrows():
    matched_title = match_document_title(scraped_row["document_title_clean"], df_documents_8)

    if matched_title:
        matched_rows.append({
            "dossier_id": scraped_row["dossier_id"],
            "document_title": matched_title,
            "document_sourcetype": scraped_row["document_sourcetype"],
            "document_date": scraped_row["document_date"]
        })

# Zet de gematchte metadata in een DataFrame
df_matched_metadata = pd.DataFrame(matched_rows)

# **Merge alleen de documenten die een match hebben (inner join)**
df_documents_8 = df_documents_8.merge(df_matched_metadata, on=["dossier_id", "document_title"], how="inner")

# ✅ **Oude kolommen overschrijven en dubbele verwijderen**
df_documents_8["document_sourcetype"] = df_documents_8["document_sourcetype_y"].combine_first(df_documents_8["document_sourcetype_x"])
df_documents_8["document_date"] = df_documents_8["document_date_y"].combine_first(df_documents_8["document_date_x"])

# ✅ **Verwijder de dubbele kolommen**
df_documents_8.drop(columns=["document_sourcetype_x", "document_sourcetype_y", "document_date_x", "document_date_y"], inplace=True)

# **Sla alleen de correcte documenten op, zonder besluitbrief.pdf**
df_documents_8.to_csv("df_documents_dossier_8.csv", index=False)

# **Weergeven in Jupyter Notebook**
display(df_documents_8)


🔍 Scrapen van dossier: 8 → https://open.minvws.nl/dossier/VWS-WC/047


,dossier_id,document_id,document_title,document_type,document_sourcetype,document_date
0,8,8-1,VWS-WC-047-7-00000148-Hulptroepen_McKinsey.pdf,pdf,Onbekend,2020-05-07
1,8,8-3,VWS-WC-047-10-00000070-PHOTO-2020-04-16-21-28-...,pdf,Onbekend,2020-04-16
2,8,8-4,VWS-WC-047-4-00000304-PHOTO-2020-04-10-23-05-4...,pdf,Onbekend,2020-04-10
3,8,8-5,VWS-WC-047-1-Totaal_overzicht_chats_Bas_vd_Dun...,pdf,PDF,2021-06-29
4,8,8-6,VWS-WC-047-3-00000539-PHOTO-2020-05-07-23-03-0...,pdf,Onbekend,2020-05-07
5,8,8-7,VWS-WC-047-5-00000305-PHOTO-2020-04-10-23-06-3...,pdf,Onbekend,2020-04-10
6,8,8-8,VWS-WC-047-6-00000085-PHOTO-2021-05-19-09-09-2...,pdf,Onbekend,2021-05-19


In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin
from IPython.display import display

# CSV-bestanden inlezen
df_dossiers = pd.read_csv("df_dossiers_final.csv")
df_documents = pd.read_csv("df_documents.csv")

# 📌 **Kies hier het dossier_id handmatig**
selected_dossier_id = 1  # Wijzig dit nummer handmatig!

# Base URL van de site
BASE_URL = "https://open.minvws.nl"

# Mapping van <use> symbolen naar documenttypen
DOCUMENT_TYPE_MAP = {
    "unknown": "Onbekend",
    "pdf": "PDF",
    "email": "E-mailbericht",
    "presentation": "Presentatie",
    "doc": "Word-document",
    "chat": "Chatbericht",
    "image": "Afbeelding",
    "spreadsheet": "Spreadsheet"
}

# Voeg kolommen toe als ze nog niet bestaan
if "document_sourcetype" not in df_documents.columns:
    df_documents["document_sourcetype"] = None
if "document_date" not in df_documents.columns:
    df_documents["document_date"] = None

# Functie om document metadata te scrapen uit een dossierpagina
def get_document_metadata(url, dossier_id):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"❌ Kan de pagina niet laden: {url} (Status: {response.status_code})")
        return []

    soup = BeautifulSoup(response.text, "html.parser")

    # Documenten verzamelen
    document_rows = soup.select("div[data-e2e-name='documents-section'] table tbody tr")

    metadata_list = []

    for row in document_rows:
        # Documentnaam ophalen en opschonen
        document_name_element = row.select_one("td div.woo-readable-width a.woo-a")
        raw_document_name = document_name_element.get_text(strip=True) if document_name_element else None
        document_name = raw_document_name.replace("_", " ") if raw_document_name else None  # Vervang underscores

        # **Besluitbrief overslaan bij scrapen**
        if document_name and "besluitbrief.pdf" in document_name.lower():
            continue  # Sla besluitbrief.pdf over

        # Documentdatum ophalen
        document_date_element = row.select_one("td time")
        document_date = document_date_element["datetime"] if document_date_element else None

        # Documenttype ophalen uit de <use> of <symbol>
        document_type_element = row.select_one("td svg use")
        document_type = None
        if document_type_element:
            type_symbol = document_type_element["xlink:href"].split("#")[-1]
            document_type = DOCUMENT_TYPE_MAP.get(type_symbol, "Onbekend")

        # Alleen opslaan als er een documentnaam is
        if document_name:
            metadata_list.append({
                "dossier_id": dossier_id,
                "document_title_clean": document_name,  # Schoon document_title (spaties i.p.v. underscores)
                "document_sourcetype": document_type,
                "document_date": document_date
            })

    return metadata_list

# **Filter alleen het geselecteerde dossier**
df_selected_dossier = df_dossiers[df_dossiers["dossier_id"] == selected_dossier_id]

# **Check of het dossier bestaat**
if df_selected_dossier.empty:
    print(f"⚠️ Geen dossier gevonden met dossier_id {selected_dossier_id}.")
else:
    dossier_url = df_selected_dossier.iloc[0]["dossier_sourceURL"]
    print(f"🔍 Scrapen van dossier: {selected_dossier_id} → {dossier_url}")

    # **Scrape de documentmetadata**
    metadata = get_document_metadata(dossier_url, selected_dossier_id)

    # **Stop als er geen documenten zijn gevonden**
    if not metadata:
        print(f"⚠️ Geen documenten gevonden voor dossier {selected_dossier_id}.")
    else:
        df_scraped_metadata = pd.DataFrame(metadata)

        # **Filter df_documents voor dit dossier en verwijder besluitbrief.pdf**
        df_documents_dossier = df_documents[
            (df_documents["dossier_id"] == selected_dossier_id) &
            (~df_documents["document_title"].str.contains("besluitbrief.pdf", case=False, na=False))
        ].copy()

        # 🔹 **Substring-matching met `_` vervangen door spaties**
        def match_document_title(scraped_title, df_docs):
            """Zoekt of `scraped_title` als substring voorkomt in `df_docs["document_title"]` (met underscores vervangen door spaties)."""
            scraped_title_clean = scraped_title.replace("_", " ")
            matches = df_docs[df_docs["document_title"].str.replace("_", " ").str.contains(scraped_title_clean, case=False, na=False)]
            if not matches.empty:
                return matches.iloc[0]["document_title"]  # Neem de eerste match
            return None  # Geen match gevonden

        # **Zoek een match voor elk gescrapet document**
        matched_rows = []
        for _, scraped_row in df_scraped_metadata.iterrows():
            matched_title = match_document_title(scraped_row["document_title_clean"], df_documents_dossier)

            if matched_title:
                matched_rows.append({
                    "dossier_id": scraped_row["dossier_id"],
                    "document_title": matched_title,
                    "document_sourcetype": scraped_row["document_sourcetype"],
                    "document_date": scraped_row["document_date"]
                })

        # Zet de gematchte metadata in een DataFrame
        df_matched_metadata = pd.DataFrame(matched_rows)

        # **Merge alleen de documenten die een match hebben (inner join)**
        df_documents_dossier = df_documents_dossier.merge(df_matched_metadata, on=["dossier_id", "document_title"], how="inner")

        # ✅ **Oude kolommen overschrijven en dubbele verwijderen**
        df_documents_dossier["document_sourcetype"] = df_documents_dossier["document_sourcetype_y"].combine_first(df_documents_dossier["document_sourcetype_x"])
        df_documents_dossier["document_date"] = df_documents_dossier["document_date_y"].combine_first(df_documents_dossier["document_date_x"])

        # ✅ **Verwijder de dubbele kolommen**
        df_documents_dossier.drop(columns=["document_sourcetype_x", "document_sourcetype_y", "document_date_x", "document_date_y"], inplace=True)

        # ✅ **Append het dossier naar `df_documenten_date.csv`**
        output_file = "df_documenten_date.csv"
        df_documents_dossier.to_csv(output_file, mode="a", header=not pd.io.common.file_exists(output_file), index=False)

        # ✅ **Laatste status printen**
        print(f"✅ Dossier {selected_dossier_id} toegevoegd aan 'df_documenten_date.csv'")


🔍 Scrapen van dossier: 1 → https://open.minvws.nl/dossier/VWS-WC/098
✅ Dossier 1 toegevoegd aan 'df_documenten_date.csv'


/tmp/ipykernel_14127/921917194.py:113: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  matches = df_docs[df_docs["document_title"].str.replace("_", " ").str.contains(scraped_title_clean, case=False, na=False)]
